In [ ]:
import json
import re
!pip install konlpy
import konlpy

test = []
for line in open('extractive_test_v2.jsonl', 'r'):
    test.append(json.loads(line))

test_article=[]

for i in test:
    test_article.append(i['article_original'])

for i in test_article:
    for j,item in enumerate(i):
        i[j]=re.compile('[^ ㄱ-ㅣ가-힣]+').sub('',item)


In [ ]:
splited_test=test_article

from konlpy.tag import Okt
okt = Okt()
for item in splited_test:
    for i,j in enumerate(item):
        item[i]=okt.morphs(j)

In [ ]:
stop_words=[]

f = open('한국어불용어100.txt', encoding="utf8")
for line in f:
    word_vector = line.split()
    stop_words.append(word_vector[0])    
f.close()

for i,item in enumerate(splited_test):
    for j,k in enumerate(item):
        splited_test[i][j]=[word for word in k if not word in stop_words]        

In [ ]:
import numpy as np
embedding_dict = dict()
f = open('glove.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()

"""
for i in range(len(test_article)):
    for j,item in enumerate(test_article[i]):
        vector=0
        for k in item:
            if k in embedding_dict:
                vector+=embedding_dict[k]
            else:
                vector+=np.zeros(100)                
        if(len(item)==0):
            test_article[i][j]=np.zeros(100) 
        else:
            test_article[i][j]=vector/len(item)
"""

In [ ]:
embedding_dim = 100
zero_vector = np.zeros(embedding_dim)

def calculate_sentence_vector(sentence):
  if len(sentence) != 0:
    return sum([embedding_dict.get(word, zero_vector) 
                  for word in sentence])/len(sentence)
  else:
    return zero_vector

In [ ]:
for i,item in enumerate(splited_test):
    for j,k in enumerate(item):
        splited_test[i][j]=calculate_sentence_vector(k)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import matplotlib.pyplot as plt

def similarity_matrix(sentence_embedding):
  sim_mat = np.zeros([len(sentence_embedding), len(sentence_embedding)])
  for i in range(len(sentence_embedding)):
      for j in range(len(sentence_embedding)):
        sim_mat[i][j] = cosine_similarity(sentence_embedding[i].reshape(1, 100),
                                          sentence_embedding[j].reshape(1, 100))[0,0]
  return sim_mat

def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

test = []
for line in open('extractive_test_v2.jsonl', 'r'):
    test.append(json.loads(line))

In [ ]:
submission=[]
for i,item in enumerate(test_article):
    print(i)
    try:
        sub=[]
        k=similarity_matrix(item)
        k=calculate_score(k)
        k=sorted(k.items(), reverse=True, key=lambda item:item[1])
        for index,(m,n) in enumerate(k):
            sub.append(test[i]['article_original'][m])
            if(index==2):
                break
        submission.append(sub)            
    
    except:
        for t in range(3):
            sub.append(test[t]['article_original'][t])
        submission.append(sub)